In [1]:
import sys, os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.signal
import importlib
import h5py

In [2]:
path = "/Users/ajc/Core/simulations/barrow_icewedges/hydrologic_connection/ProjectionsO/"
var_set = ['surface_flux-day.dat','vertical_slice_rim-mass_flux100cm.dat']

outfile='/Users/ajc/Documents/MyPapers/TroughHydrodynamics/figures/Dec22_uniform_om/'

In [3]:
#Dec 22, 2020: ProjectionsO
Simulations = [['CESM_C2%s'%i for i in range(3)], ['CESM_C1%s'%i for i in range(4)],['CESM_C3%s'%i for i in range(3)] ]# modified rain model, dessicated zone =0.1, longwaves



In [4]:
def read_file(file):
    d = np.loadtxt(file,usecols=1,skiprows=6)
    t1 = np.loadtxt(file,usecols=0,skiprows=6)
    t = np.array([x/86400./365. for x in t1])
    #print (len(d),len(t))
    l = np.minimum(len(d),len(t))
    return d[:l],t[:l]

def read_set(set,var):
    dat = []
    tim = []
    for s in set:
        file = os.path.join(path,s, var)
       
        d = np.loadtxt(file,usecols=1,skiprows=6)
        t1 = np.loadtxt(file,usecols=0,skiprows=6)
        t = np.array([x/86400./365. for x in t1])
        

        L = len(d)%365
        if L >0:
            #print ('LL: ',L)
            d = d[:-L]
            t = t[:len(d)]
        t0 =0
        if (len(tim)==0):
            dat.append(d)
            tim.append(t)
        else:
            
            t0 = np.where(t <=tim[-1][-1])[0]
            
            if len(t0) >0:
                d = d[t0[-1]:]
                t = t[t0[-1]:]
                #print ('T0: ',s,t, tim[-1][-1],t0)
                #print (s, t[0],tim[-1][-1])
            dat.append(d)
            tim.append(t)
        

   
    dat = np.concatenate(dat)
    tim = np.concatenate(tim)
    L = len(dat)%365
    if L >0:
        dat = dat[:-L]
        tim = tim[:-L] 
    #print ('Lengths: ',L, len(dat)%365, len(tim)%365,len(dat), len(tim), len(dat))
    
    return dat, tim


In [5]:
colors = ['r','k','g','m','c','y']
marks = ['.','*', '+', 'o']
marks = ['-','-','-']
Labels = [r'S$_{1}$', r'S$_{2}$', r'S$_{3}$']

In [16]:
forcing_watershed_FM = h5py.File("/Users/ajc/Core/PreProcessing/ats-intermediate-ngee/data/SNAP/CESM-RCP8.5-2006-2100_dm1985-2015-FM.h5",'r')

d1 = forcing_watershed_FM['precipitation snow [m SWE s^-1]'][:] 
dsnow = [-x*86400*1000*365 for x in d1]

d1 = forcing_watershed_FM['precipitation rain [m s^-1]'][:] 
drain = [-x*86400*1000*365 for x in d1]

d1 = forcing_watershed_FM['time [s]']
d3_time = [t/86400/365. for t in d1]

forcing_watershed_FM.close()
print (np.mean(dsnow[:365]))

-161.55121004074672


In [7]:
def get_moving_average(d, m=5):
    d_avg = []
    len1 = len(d)
    for i in range(len1):
        if i < m:
            s1 = np.mean(d[:i+1])
        else:
            s1= np.mean(d[i-m:i])

        d_avg.append(s1)
    return d_avg

In [8]:
%matplotlib qt
to_day_year = 86400 *365
Colors = ['b','k','r','r']
Labels = ['zero-head','specified-head','dynamic-head']

surface_area = 61.92
from matplotlib import gridspec
d_snowmelt = []
d_postsnowmelt = []
d_S3 =[]
d_S1 =[]
def get_DischargeXX():
    fig, axss = plt.subplots(5,1, figsize=(5,7), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    gs = gridspec.GridSpec(5, 1)#, width_ratios=[1]) 
    plt.tight_layout()
    
    time = 2006 + np.linspace(0,94,94,dtype='int')
    axs = plt.subplot(gs[1:3])
    
    for i,sim in enumerate(Simulations):
        d1, t1 = read_set(sim,'surface_flux-day.dat')
        d11 = [x/(55555*surface_area)*to_day_year*1000 for x in d1] #converted to day here
        #print (d11[:200])
        time = 2005 + np.reshape(t1,(-1,365)).mean(axis=1)
        if (i==2):
            d12 = np.reshape(d11,(-1,365))
            d21 = []
            for d in d12:
                dx = np.where(d>=0)
                d_S3.append(np.sum(d[dx])/len(d))
                dx1 = d[dx][:181]
                sum1 = np.sum(dx1)/len(d)
                d21.append(sum1)
                
            d21 = get_moving_average(d21)
            axs.plot(time, d21,color=Colors[2])#,label=Labels[2])
            d_snowmelt.append((i,d21))
            
            f = np.polyfit(time,d21,1)
            p = np.poly1d(f)
            axs.plot(time,p(time),color=Colors[2],linestyle='dashed')
            
        else:
            d2 = np.reshape(d11,(-1,365)).mean(axis=1)#*365
            print (len(d2))
            #if (i==0):
            #    d_S1.append(np.sum(d[dx])/len(d))
            d21 = d2[:181]
            d21 = get_moving_average(d21)
        
            axs.plot(time, d21,color=Colors[i])#,label=Labels[i])
            f = np.polyfit(time,d21,1)
            p = np.poly1d(f)
            axs.plot(time,p(time),color=Colors[i],linestyle='dashed')
            d_snowmelt.append((i,d21))
        axs.set_xlim((2005.5, 2100))
        axs.set_ylim((0,299.9))
        axs.set_xlabel('Year',fontsize=12, fontweight='normal')
        
        axs.set_ylabel('Q [mm]',fontsize=12, fontweight='normal')
        #axs.legend(loc='lower left',labelspacing=.1)
        #axs.axhline(y=0,linestyle='dashed',color='b')
        axs.text(2006,280,'(b) during snowmelt')
        axs.xaxis.set_visible(False)
        
        # Rain
        ax1 = plt.subplot(gs[0])
        force_time = np.reshape(d3_time,(-1,365)).mean(axis=1)
        force_rain = -np.reshape(dsnow,(-1,365)).mean(axis=1)
        ax1.plot(2005+force_time,force_rain,'grey',linestyle='-')
        ax1.set_ylim([1, 300])
        ax1.xaxis.set_visible(False)
        ax1.set_xlim((2005., 2100))
        gs.update(wspace=0.025, hspace=0.05)
        
        ax1.set_ylabel(r'P$_{\mathrm{snow}}$ [mm]',fontsize=12,color='k')
        
        f = np.polyfit(force_time,force_rain,1)
        p = np.poly1d(f)
        ax1.plot(2006+force_time,p(force_time),color='k',linestyle='dashed')
        ax1.text(2006,250,'(a)')
        
    axs = plt.subplot(gs[3:5])
    
    for i,sim in enumerate(Simulations):
        d1, t1 = read_set(sim,'surface_flux-day.dat')
        d11 = [x/(55555*surface_area)*to_day_year*1000 for x in d1] #converted to day here
        time = 2005 + np.reshape(t1,(-1,365)).mean(axis=1)
        if (i==2):
            d12 = np.reshape(d11,(-1,365))
            d21 = []
            for d in d12:
                dx = np.where(d>=0)
                dx1 = d[dx][182:]
                sum1 = np.sum(dx1)/len(d)
                d21.append(sum1)
                
            d21 = get_moving_average(d21)
            axs.plot(time, d21,color=Colors[2],label=Labels[2])
            d_postsnowmelt.append((i,d21))
            f = np.polyfit(time,d21,1)
            p = np.poly1d(f)
            axs.plot(time,p(time),color=Colors[2],linestyle='dashed')
        else:
            d2 = np.reshape(d11,(-1,365))
            d21 = []
            for d in d2:
                #print (len(d))
                dxx = np.array(d[182:])
                d21.append(np.sum(dxx)/365.)
            #print (d3)
            d21 = get_moving_average(d21)
            #print (len(d3),d3)
            axs.plot(time, d21,color=Colors[i],label=Labels[i])
            f = np.polyfit(time,d21,1)
            p = np.poly1d(f)
            axs.plot(time,p(time),color=Colors[i],linestyle='dashed')
            d_postsnowmelt.append((i,d21))
        axs.set_xlim((2005.5, 2100))
        axs.set_ylim((0,99.))
        axs.set_xlabel('Year',fontsize=12, fontweight='normal')
        
        axs.set_ylabel('Q [mm]',fontsize=12, fontweight='normal')
        axs.legend(loc='upper right',labelspacing=.1)
        #axs.axhline(y=0,linestyle='dashed',color='b')
        axs.text(2006,93,'(c) post-snowmelt')
        #axs.xaxis.set_visible(False)
        
    #plt.savefig(outfile+'annual-Q-split.pdf', bbox_inches='tight', dpi=500)
    #return Discharge,Recharge

In [47]:
%matplotlib qt
to_day_year = 86400 *365
Colors = ['b','k','r','r']
Labels = ['zero-head','specified-head','dynamic-head']

surface_area = 61.92
from matplotlib import gridspec
d_snowmelt = []
d_postsnowmelt = []
d_S3 =[]
d_S1 =[]
def get_Discharge():
    fig, axss = plt.subplots(5,1, figsize=(5,7), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    gs = gridspec.GridSpec(5, 1)#, width_ratios=[1]) 
    plt.tight_layout()
    
    time = 2006 + np.linspace(0,94,94,dtype='int')
    axs = plt.subplot(gs[1:3])
    
    for i,sim in enumerate(Simulations):
        d1, t1 = read_set(sim,'surface_flux-day.dat')
        d11 = [x/(55555*surface_area)*to_day_year*1000 for x in d1] #converted to day here
        #print (d11[:200])
        time = 2005 + np.reshape(t1,(-1,365)).mean(axis=1)
       
        d12 = np.reshape(d11,(-1,365))
        d21 = []
        for d in d12:
            dx = np.where(d>0)
            #print ('snowmelt',i, dx)
            dx1 = d[dx][:181]
            sum1 = np.sum(dx1)/len(d)
            d21.append(sum1)

        d21 = get_moving_average(d21)
        axs.plot(time, d21,color=Colors[i])#,label=Labels[2])
        d_snowmelt.append((i,d21))

        f = np.polyfit(time,d21,1)
        p = np.poly1d(f)
        axs.plot(time,p(time),color=Colors[i],linestyle='dashed')
        #d_S3.append(np.sum(d[dx])/len(d))

        axs.set_xlim((2005.5, 2100))
        axs.set_ylim((0,299.9))
        axs.set_xlabel('Year',fontsize=12, fontweight='normal')
        
        axs.set_ylabel('Q [mm]',fontsize=12, fontweight='normal')
        #axs.legend(loc='lower left',labelspacing=.1)
        #axs.axhline(y=0,linestyle='dashed',color='b')
        axs.text(2008,280,'Early summer')
        axs.xaxis.set_visible(False)
        
    # Rain
    ax1 = plt.subplot(gs[0])
    force_time = np.reshape(d3_time,(-1,365)).mean(axis=1)
    force_snow = -np.reshape(dsnow,(-1,365)).mean(axis=1)
    force_rain = -np.reshape(drain,(-1,365)).mean(axis=1)
    ax1.plot(2005+force_time,force_rain,'green',linestyle='-',label='Rain')
    ax1.plot(2005+force_time,force_snow,'grey',linestyle='-',label='Snow')

    ax1.set_ylim([1, 300])
    ax1.xaxis.set_visible(False)
    ax1.set_xlim((2005., 2100))
    gs.update(wspace=0.025, hspace=0.05)

    ax1.set_ylabel(r'P [mm]',fontsize=12)

    f = np.polyfit(force_time,force_snow,1)
    p = np.poly1d(f)
    ax1.plot(2006+force_time,p(force_time),color='grey',linestyle='dashed')
    #ax1.text(2006,250,'(a)')
    f = np.polyfit(force_time,force_rain,1)
    p = np.poly1d(f)
    ax1.plot(2006+force_time,p(force_time),color='green',linestyle='dashed')
    ax1.legend(loc='upper left',ncol=2,fontsize=6,bbox_to_anchor=[0.0, 1.])
    #ax1.legend(loc='upper left', ncol=2,labelspacing=.01,fontsize=8)
    #ax1.text(2006,250,'(a)')
    axs = plt.subplot(gs[3:5])
    
    for i,sim in enumerate(Simulations):
        d1, t1 = read_set(sim,'surface_flux-day.dat')
        d11 = [x/(55555*surface_area)*to_day_year*1000 for x in d1] #converted to day here
        time = 2005 + np.reshape(t1,(-1,365)).mean(axis=1)
        if (i>-1):
            d12 = np.reshape(d11,(-1,365))
            d21 = []
            for d in d12:
                dx = np.where(d>=0)
                dx1 = d[dx][182:]
                sum1 = np.sum(dx1)/len(d)
                d21.append(sum1)
                
            d21 = get_moving_average(d21)
            axs.plot(time, d21,color=Colors[i],label=Labels[i])
            d_postsnowmelt.append((i,d21))
            f = np.polyfit(time,d21,1)
            p = np.poly1d(f)
            axs.plot(time,p(time),color=Colors[i],linestyle='dashed')
        else:
            d2 = np.reshape(d11,(-1,365))
            d21 = []
            for d in d2:
                #print (len(d))
                dxx = np.array(d[182:])
                d21.append(np.sum(dxx)/365.)
            #print (d3)
            d21 = get_moving_average(d21)
            #print (len(d3),d3)
            axs.plot(time, d21,color=Colors[i],label=Labels[i])
            f = np.polyfit(time,d21,1)
            p = np.poly1d(f)
            axs.plot(time,p(time),color=Colors[i],linestyle='dashed')
            d_postsnowmelt.append((i,d21))
        axs.set_xlim((2005.5, 2100))
        axs.set_ylim((0,99.))
        axs.set_xlabel('Year',fontsize=12, fontweight='normal')
        
        axs.set_ylabel('Q [mm]',fontsize=12, fontweight='normal')
        axs.legend(loc='upper right',labelspacing=.1)
        #axs.axhline(y=0,linestyle='dashed',color='b')
        axs.text(2008,93,'Late summer')
        #axs.xaxis.set_visible(False)
        
    plt.savefig(outfile+'annual-Q-split.pdf', bbox_inches='tight', dpi=500)
    #return Discharge,Recharge

In [48]:
get_Discharge()

In [13]:
31+28+31+30+31+20

171

In [ ]:
d1 = d_snowmelt[0][1] + d_postsnowmelt[0][1]
d2 = d_snowmelt[1][1] + d_postsnowmelt[1][1]
d3 = d_snowmelt[2][1] + d_postsnowmelt[2][1]
plt.plot(d1,'b')
plt.plot(d2,'k')
plt.plot(d3,'r')